In [1]:
import json
from ezprettyjson import prettyjson

In [2]:
files = []
for fname in map(lambda s: s+'.json', ['ng_catalog'
                                      ,'bodge_cat_snap'
                                      ,'bodge_catalog'
                                      ,'bodge_cat_160414'
                                      ,'bodge_catalog-final'
                                      ,'ng_catalog_backup160503'
                                      ,'models/ng_catalog'
                                      ,'ml/batch50-nodrop'
                                      ,'ml/batch50-nogauss']):
    with open(fname, 'r') as f:
        files.append(json.load(f))

In [3]:
all_keys = {k for f in files for k in f.keys()}

In [4]:
lens_per_key = {k: [len(v) for f in files for fk,v in f.iteritems() if fk == k] for k in all_keys}

In [5]:
conflict_keys = map(lambda t: t[0], 
                    filter(lambda (_,v): len(v)>1 and not all(map(lambda e: e==v[0], v)), lens_per_key.iteritems()))

In [24]:
prettyjson({k: [d[k] for d in files if k in d] for k in conflict_keys})

In [7]:
def take_if_completer(accum, new):
    completer = True # assume this because the files are ordered by date
    count_finished = lambda v: sum(1 if (hasattr(vk, '__getitem__') and 'finished' in vk and vk['finished']) else 0 for vk in v.itervalues())
    if not new[0] in accum:
        accum[new[0]] = new[1]
        return accum # Take unseen things regardless
    
    if new[0].startswith('m-') and accum[new[0]]['pretrain_start_time']>new[1]['pretrain_start_time']:
        #Always take the later model
        completer = False        
    elif new[0].startswith('e-'): #encdecs
        pass #There's no way to find out which encdecs are more recent so always take the new
    else: #summary or best/latest
        # If a summary has more entries it's more recent
        # If the amount is equal but more are completed it's also more recent 
        if len(accum[new[0]]) < len(new[1]):
            completer = False
        elif count_finished(accum[new[0]]) > count_finished(new[1]):
            completer = False

    if completer:
        accum[new[0]] = new[1]
    return accum

In [8]:
merged = reduce(take_if_completer, [(k,v) for d in files for k,v in d.iteritems()], {})

In [9]:
#prettyjson({k:v for k,v in merged.iteritems() if k.startswith('m-final-test-off')})

In [10]:
#prettyjson({k:v for k,v in merged.iteritems() if k.startswith('e-final-test-off')})

In [11]:
inconsistent_keys = {k for k,v in merged.iteritems() 
                     if len({sk for sk in merged.iterkeys() 
                             if sk.startswith('m-'+k)}) !=
                        len({vk for vk in v.iterkeys()
                             if vk.startswith('-')})
                     and k != "best" and k != "latest"}

In [12]:
inconsistents = {k: merged[k] for k in inconsistent_keys}

In [13]:
merged_n = inconsistents
for k in inconsistent_keys:
    models = {mk:mv for mk,mv in merged.iteritems() if mk.startswith('m-'+k) and len(mk) <= len(k)+14}
    mvals = [(mv['pretrain_start_time'], mv['pretrain_end_time'], mv['test_accuracy'], mk)
             for mk,mv in models.iteritems()]
    summaries = {n:{"finished": True
                   ,"start_time": mval[0]
                   ,"end_time": mval[1]
                   ,"test_accuracy": mval[2]}
                 for mval in mvals for n in (mval[3][2+len(k):],)}
    merged_n[k].update(summaries)
prettyjson(merged_n, depth=2, length=20)

In [14]:
print(len(merged))
merged.update(merged_n)
print(len(merged))

614
614


In [15]:
import functools
def compose(*functions):
    return functools.reduce(lambda f, g: lambda x: f(g(x)), functions, lambda x: x)

In [22]:
# Remove rubbish
rubbish_conds = [lambda k: False if k and k.startswith('e-e-') else k
                ,lambda k: False if k and 'best' in k else k
                ,lambda k: False if k and 'latest' in k else k
                ,lambda k: False if k and 'test-160406' in k else k
                ,lambda k: False if k and 'runner-test' in k else k
                ,lambda k: False if k and k.startswith('ed-e') else k]
rcond = compose(*rubbish_conds)
merged_w = {k:v for k,v in merged.iteritems() if rcond(k)}

In [23]:
from ml.catalog import CustomJSONEncoder
with open('catalog-merged-fixed-plus_runner_test.json', 'w') as f:
    f.write(json.dumps(merged_w, indent=2, sort_keys=True, cls=CustomJSONEncoder))